## TAAS 교통사고데이터 좌표 크롤러 with Selenium
Selenium을 이용한 동적 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from urllib import parse
import time
import pandas as pd
import re
from tqdm import tqdm

In [ ]:
# TAAS GIS 시스템 로드
driver = webdriver.Chrome('/Users/dangchan/Documents/chromedriver')
driver.maximize_window()
driver.get('http://taas.koroad.or.kr/gis/mcm/mcl/initMap.do?menuId=GIS_GMP_STS_RSN')
time.sleep(1)

<ipython-input-116-468e68a7da69>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/dangchan/Documents/chromedriver')


In [ ]:
# 체크박스 등 PM만 체
elements = driver.find_elements(By.NAME, 'ACDNT_GAE_CODE')[0:4]
for element in elements:
    if not element.is_selected():
        element.click()

selectbox = driver.find_element(By.ID, 'ptsRafSigungu')
select = Select(selectbox)
select.select_by_value("01") # 전체 지역구(서울)

vehicle = driver.find_element(By.XPATH, '//*[@id="ptsRaf-WRNGDO_VHCLE_ASORT_CODE"]/a') # 가해차종 > PM만 체크
vehicle.click()
time.sleep(1)

pm = driver.find_element(By.XPATH, '//*[@id="ptsRafCh1PerpetratorVehicleType"]/li[9]/input')
if not pm.is_selected():
    pm.click()

In [ ]:
# search
driver.find_element(By.XPATH, '//*[@id="regionAccidentFind"]/div[2]/p/a').send_keys(Keys.ENTER)
time.sleep(1)

In [ ]:
# Crawling
coord_ls = []
srs_ls = []

for i in tqdm(range(0,445)):
    driver.execute_script(f'gis.srh.msh.selectListUi({i}, 11);') # 개별 사고지점
    time.sleep(1) # 로드
    bbox = driver.find_elements(By.CLASS_NAME,'olTileImage')[-1]
    src = parse.unquote(bbox.get_attribute('src')) # get src and unquote from 16byte
    src_split = src.split('&')
    bbox_text = src_split[-3][5:]
    coords = [float(i) for i in bbox_text.split(',')]
    x, y = (coords[0]+coords[2])/2, (coords[1]+coords[3])/2 # center position
    srs = src_split[-4][4:]
    coord_ls.append([x, y])
    srs_ls.append(srs)

100%|██████████| 445/445 [09:38<00:00,  1.30s/it]


In [ ]:
# 데이터프레임 병합
accident_df = pd.read_csv('accidentInfoList.csv')
accident_df['좌표'] = coord_ls
accident_df['좌표계'] = srs_ls
accident_df.to_csv('accident_df.csv',index=False)

In [ ]:
# 최종 데이터프레임
accident_df

,사고번호,사고일시,요일,시군구,도로명주소,사고내용,사망자수,중상자수,경상자수,부상신고자수,...,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,좌표,좌표계
0,2021010200100053,2021년 1월 2일 10시,토요일,서울특별시 마포구 구수동,NaN,중상사고,0,1,0,0,...,개인형이동수단(PM),남,28세,상해없음,이륜,남,49세,중상,"[194066.02579262, 449747.90669503]",EPSG:5181
1,2021011000100182,2021년 1월 10일 16시,일요일,서울특별시 서초구 반포동,NaN,경상사고,0,0,1,0,...,개인형이동수단(PM),남,24세,경상,승용,남,43세,상해없음,"[201601.91336089, 445614.34964187]",EPSG:5181
2,2021011400100148,2021년 1월 14일 10시,목요일,서울특별시 서초구 서초동,NaN,부상신고사고,0,0,0,1,...,개인형이동수단(PM),여,24세,부상신고,승용,여,39세,상해없음,"[200524.55963108, 442588.44509583]",EPSG:5181
3,2021011500100555,2021년 1월 15일 21시,금요일,서울특별시 송파구 송파동,NaN,경상사고,0,0,1,0,...,개인형이동수단(PM),남,26세,상해없음,보행자,여,30세,경상,"[209569.1932179, 445214.50896024]",EPSG:5181
4,2021011700100198,2021년 1월 17일 18시,일요일,서울특별시 강남구 논현동,NaN,경상사고,0,0,1,0,...,개인형이동수단(PM),남,34세,상해없음,보행자,여,48세,경상,"[202131.5358183, 446477.50152318]",EPSG:5181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,2021122800100515,2021년 12월 28일 22시,화요일,서울특별시 종로구 종로1가,NaN,중상사고,0,1,0,0,...,개인형이동수단(PM),남,47세,중상,승합,남,59세,상해없음,"[197999.79775891, 452332.8224699]",EPSG:5181
441,2021122900100086,2021년 12월 29일 09시,수요일,서울특별시 중랑구 상봉동,NaN,부상신고사고,0,0,0,1,...,개인형이동수단(PM),남,24세,부상신고,승용,남,31세,상해없음,"[207293.73121796, 455099.3581697]",EPSG:5181
442,2021122900100089,2021년 12월 29일 09시,수요일,서울특별시 강남구 청담동,NaN,경상사고,0,0,1,0,...,개인형이동수단(PM),남,36세,경상,승용,남,44세,상해없음,"[204325.47183565, 446840.30029671]",EPSG:5181
443,2021123000100141,2021년 12월 30일 11시,목요일,서울특별시 양천구 신월동,NaN,중상사고,0,1,0,0,...,개인형이동수단(PM),남,62세,중상,승용,남,67세,상해없음,"[184868.76443644, 448577.57359732]",EPSG:5181
